In [2]:
pip install polyline

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests, json, time
import polyline, folium
import datetime
import re

In [6]:
with open('config.json') as f:
    config = json.load(f)
    key = config['apiKey']
    
mode = 'transit'
origin = 'Oqium'
destination = 'Oallery'
departure = "09/03/2022 12:48"
arrival = None
   
def map_start(a, b):
    return ((a[0] + b[0]) / 2, (a[1] + b[1]) / 2)
    
def convert_unix(date):
    if date != None:
        return int(time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y %H:%M").timetuple()))
    else:
        return None
    
def json_route(origin, destination, departure, arrival):
    url = 'https://maps.googleapis.com/maps/api/directions/json?'
    params = {
        'mode': mode.lower(),
        'origin': origin,
        'destination':destination, 
        'region':'nl',
        'key': key
    }
    if departure != None:
        params['departure_time'] = convert_unix(departure)
    
    response = requests.post(url, params=params)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return {}

def get_coordinates(raw_json):
    if len(raw_json) > 0:
        coordinates = polyline.decode(raw_json['routes'][0]['overview_polyline']['points'], 5)
        return coordinates
    else:
        return False

def make_map(coordinates):
    if coordinates != False:
        start_coord = coordinates[0]
        end_coord = coordinates[len(coordinates) - 1]

        map = folium.Map(location=map_start(start_coord, end_coord),
                    zoom_start=15)
        
        folium.PolyLine(coordinates,
                        color='red',
                        weight=5,
                        opacity=0.8).add_to(map)
        
        folium.Marker(location=start_coord, popup=f'Start Location - <b>{origin}</b>', tooltip=f'<strong>{origin}</strong>', icon=folium.Icon(color='red', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=end_coord, popup=f'End Location - <b>{destination}</b>', tooltip=f'<strong>{destination}</strong>', icon=folium.Icon(color='red', icon='flag-checkered', prefix = 'fa')).add_to(map)
        map.save("map.html")
        display(map)
    else:
        return "Could not process map"
        
def process_routing_info(raw_json):
    steps = []
    distance = raw_json['routes'][0]['legs'][0]['distance']['value']
    duration = raw_json['routes'][0]['legs'][0]['duration']['text']
    
    if mode in ['transit']:
        departure_time = raw_json['routes'][0]['legs'][0]['departure_time']['text']
        arrival_time = raw_json['routes'][0]['legs'][0]['arrival_time']['text']
        times = (departure_time, arrival_time)
        print(f"Planned departure: {departure_time}\nPlanned arrival: {arrival_time}")
    else:
        times = None
    
    if distance >= 1000:
        distance = round(distance/1000, 2)
        print('Distance: {} km'.format(distance))
    else:
        print('Distance {} m'.format(distance))
    print(f"Duration: {duration}")
    
    steps = [re.sub('<[^<]+?>', '', step['html_instructions'].replace('<div style="font-size:0.9em">', '\n')) for step in raw_json['routes'][0]['legs'][0]['steps']]
    for step in range(len(steps)):
        print(f"{step+1}. {steps[step]}")
    return [distance, duration, steps, times]

def main():
    raw_json = json_route(origin, destination, departure, arrival)
    #print(raw_json)
    #display(raw_json)
    coordinates = get_coordinates(raw_json)
    print(coordinates)
    make_map(coordinates)
    route_info = process_routing_info(raw_json)
    
main()


[(52.3717, 4.89048), (52.37178, 4.8905), (52.3719, 4.89047), (52.37216, 4.89046), (52.37217, 4.89059), (52.37238, 4.89059), (52.37251, 4.89049), (52.37254, 4.89048), (52.37254, 4.89066), (52.37257, 4.89084), (52.37274, 4.89084), (52.37286, 4.89083), (52.37315, 4.89077), (52.37325, 4.89076), (52.37354, 4.89088), (52.3736, 4.89091), (52.37374, 4.89095), (52.37377, 4.89078), (52.37375, 4.89083), (52.3735, 4.89071), (52.37319, 4.89059), (52.37309, 4.89056), (52.37252, 4.8906), (52.37189, 4.89063), (52.37159, 4.89065), (52.37148, 4.89065), (52.37137, 4.89061), (52.37094, 4.89038), (52.37075, 4.89026), (52.37051, 4.89003), (52.37028, 4.88983), (52.37016, 4.88975), (52.36978, 4.88952), (52.36929, 4.8893), (52.36885, 4.88926), (52.36877, 4.88924), (52.36866, 4.88918), (52.36857, 4.88901), (52.36839, 4.8888), (52.36824, 4.88876), (52.36816, 4.8888), (52.36764, 4.88946), (52.36756, 4.88955), (52.36744, 4.88962), (52.36739, 4.88962), (52.36732, 4.8896), (52.36721, 4.88949), (52.36711, 4.8893), (5

Planned departure: 12:50pm
Planned arrival: 1:02pm
Distance: 1.65 km
Duration: 12 mins
1. Walk to Amsterdam, Dam
2. Tram towards Nieuw Sloten
3. Walk to Nieuwe Spiegelstraat 39, 1017 DC Amsterdam, Netherlands


In [30]:
coordinates =[[52.36241,4.86427], [52.36243,4.86435],[52.36244,4.8644],
          [52.3625,4.86462],[52.36264,4.86518],[52.36276,4.86566],
          [52.36288,4.86619],[52.36307,4.86694],[52.3631,4.86707],
          [52.36312,4.86706],[52.36322,4.86704],[52.36332,4.86697],
          [52.36342,4.8669],[52.36359,4.86677],[52.36364,4.86672],
          [52.36357,4.86647],[52.36357,4.86643],[52.36353,4.86633],
          [52.36351,4.86624],[52.36344,4.86596],[52.3634,4.86577],
          [52.36331,4.86541],[52.3633,4.86538],[52.36324,4.86513],
          [52.36322,4.86501],[52.36314,4.86472],[52.36307,4.86443],
          [52.36296,4.86398],[52.36289,4.86373],[52.36286,4.86358],
          [52.36283, 4.86345],[52.36278, 4.86332],[52.36271, 4.86304],
          [52.36258, 4.86251],[52.36248, 4.86209],[52.36243,4.86188],
          [52.36221, 4.86094],[52.36208, 4.86039],[52.36204, 4.86022],
          [52.36219, 4.86012],[52.36228, 4.86007],[52.36227, 4.85995],
          [52.36217, 4.86002],[52.36203, 4.8601],[52.36204, 4.86022],
          [52.36219, 4.86012],[52.36228, 4.86007],[52.36235, 4.86003],
          [52.36232, 4.85992],[52.36232, 4.85982],[52.36229, 4.85972],
          [52.3622, 4.85933],[52.36208, 4.85884],[52.36199, 4.85846],
          [52.36197, 4.85837],[52.36195, 4.85827],[52.36195, 4.85825],
          [52.36195, 4.85824],[52.36196, 4.85824],[52.36196, 4.85823],
          [52.36197, 4.85822],[52.36198, 4.85822],[52.36199, 4.85821], 
          [52.362, 4.85822],[52.36201,4.85823],[52.36204, 4.85824], 
          [52.36208, 4.85828],[52.36219, 4.85838],[52.36266, 4.85879],
          [52.36268, 4.85871],[52.36275, 4.85874],[52.3628, 4.85877],
              [52.36286, 4.85881],[52.36308, 4.859],[52.36334, 4.85923],
[52.3634, 4.85927],[52.36351, 4.85932],[52.36388, 4.85964],
[52.36393, 4.85969],[52.36424, 4.85995],[52.36432, 4.86002],
[52.36438, 4.86011],[52.36445, 4.86016],[52.36446, 4.86013],
[52.36451, 4.85997],[52.36452, 4.85994],[52.36453, 4.85992],
[52.36457, 4.85957],[52.36457, 4.8595],[52.36464, 4.85947],
[52.36473, 4.85947],[52.36481, 4.85947],[52.36492, 4.85935],

[52.36486, 4.85964],[52.36484, 4.85967],[52.36478, 4.85982],
[52.36472, 4.85987],[52.36471, 4.85991],[52.36465, 4.86006],
[52.36465, 4.86009],[52.36459, 4.86024],[52.36457, 4.86031],
[52.36464, 4.86038],[52.36474, 4.8604],[52.36487, 4.86051],
[52.36495, 4.86058],[52.36512, 4.86073],[52.36518, 4.86079],
[52.3654, 4.86099],[52.36548, 4.86106],[52.36554, 4.86115],
[52.36559, 4.86116],[52.36602, 4.86154],[52.36606, 4.86158],
[52.36674, 4.8622],[52.36683, 4.86227],[52.36698, 4.8624],
[52.3672, 4.86261],[52.36729, 4.86269],[52.36732, 4.86271],
[52.36756, 4.86292],[52.36761, 4.863],[52.36767, 4.86305],
[52.36776, 4.86313],[52.36783, 4.86317],[52.368, 4.86331],
[52.36801, 4.86333],[52.36804, 4.86335],[52.3681, 4.86344],
[52.3682, 4.86348],[52.36822, 4.86355], [52.36823, 4.86368],


[52.36824, 4.86378],[52.36829, 4.86398],[52.3683, 4.86422],
[52.36841, 4.86445],[52.36849, 4.86479],[52.3687, 4.86467],
[52.36875, 4.86463],[52.36881, 4.86488],[52.36888, 4.86514],
[52.36889, 4.8652],[52.36893, 4.86536],[52.36892, 4.86541],
[52.36892, 4.86545],[52.36892, 4.86549],[52.36893, 4.86555],
[52.36894, 4.86561],[52.36897, 4.8657],[52.36891, 4.86572],
[52.36882, 4.86576],[52.36873, 4.86584],[52.36867, 4.86586],
[52.36861, 4.86573],[52.36849, 4.86506],[52.36809, 4.86523],
[52.36799, 4.86528],[52.36792, 4.86529],[52.36795, 4.86526],
[52.36786,4.86532], [52.36787, 4.86537],[52.36794, 4.86564],

[52.36797, 4.86576],[52.36801, 4.86593],[52.36804, 4.86608],
[52.36783, 4.86622],[52.36768, 4.86633],[52.3673, 4.86657],
[52.36719, 4.86664],[52.36652, 4.8671],[52.3664, 4.86717],
[52.36635, 4.86721],[52.36639, 4.86738],[52.36657, 4.86802],
[52.36668, 4.86844],[52.36682, 4.86896],[52.36685, 4.86908],
[52.36687, 4.86914],[52.3668, 4.86917],[52.36673, 4.86923],
[52.3666, 4.86933],[52.36654, 4.86939],[52.36625, 4.86831],
[52.36621, 4.86806],[52.36616, 4.86793],[52.36607, 4.86759],
[52.36603, 4.86745],[52.36586, 4.86758],[52.36559, 4.86781],
[52.36556, 4.86783],[52.3655, 4.86784],[52.36539, 4.86786],
[52.36534, 4.86761],[52.3652, 4.86699],[52.36513, 4.86684],
[52.36498, 4.86617],[52.36496, 4.866],[52.36486, 4.86606],
[52.36451, 4.86627],[52.36447, 4.86631],[52.36434, 4.86635],
[52.36419, 4.86643],[52.36412, 4.86649],[52.36408, 4.86651],
[52.36402, 4.8666],[52.36399, 4.86663],[52.36394, 4.86667],

[52.36396, 4.86675],[52.36397, 4.86678],[52.36397, 4.86697],
[52.36404, 4.86727],[52.3641, 4.86751],[52.36418, 4.8677],
[52.36418, 4.86772],[52.36425, 4.86802],[52.36426, 4.86813],
[52.3643, 4.86832],[52.36432, 4.8684],[52.36437, 4.86852],
[52.36444, 4.86873],[52.36443, 4.86883],[52.36445, 4.86888],
[52.36459, 4.86945],[52.36477, 4.87015],[52.36487, 4.8705],
[52.3649, 4.87056],[52.36491, 4.87067],[52.36492, 4.87071],
[52.36493, 4.87075],[52.36511, 4.8714],[52.36518, 4.87161],
[52.36522, 4.87164],[52.36517, 4.87168],[52.36511, 4.87173],
[52.3651, 4.87174],[52.36505, 4.87178],[52.36501, 4.87181],
[52.36496, 4.87185],[52.36486, 4.87183],[52.3648, 4.87188],
[52.36473, 4.87203],[52.36468, 4.87207],[52.36463, 4.87211],
[52.36457, 4.87215],[52.3645, 4.87219],[52.36441, 4.87212],
[52.36437, 4.87214],[52.36417, 4.87228],[52.36415, 4.8723],
[52.36411, 4.87233],[52.36409, 4.87235],[52.36386, 4.87249],
[52.36384, 4.87251],[52.36347, 4.87277],[52.36346, 4.87278],
[52.36342, 4.87279],[52.36339, 4.87281],[52.36301, 4.87308],
[52.36302, 4.87308],[52.36293, 4.87315],[52.36274, 4.87329],
[52.36265, 4.87335],[52.36249, 4.87345],[52.36244, 4.87349],
[52.36237, 4.87327],[52.36236, 4.87321],[52.36232, 4.87307],
[52.3623, 4.87299],[52.36218, 4.87254],[52.36213, 4.87235],
[52.36196, 4.8717],[52.36193, 4.87156],[52.36189, 4.87142],
[52.36182, 4.87119],[52.36177, 4.87101],[52.36172, 4.87087],
[52.36164, 4.87058],[52.36153, 4.87017],[52.36147, 4.86994],
[52.36139, 4.86962],[52.36138, 4.86958],[52.36134, 4.86943],
[52.3613, 4.86928],[52.36129, 4.86923],[52.36125, 4.86909],
[52.36122, 4.86898],[52.36115, 4.86871],[52.36113,4.86862],
[52.36111, 4.86862],[52.3611,4.86849],[52.36103, 4.86823],
[52.36097, 4.86798],[52.36096, 4.8679],[52.36094, 4.86784],
[52.3609, 4.86769],[52.36087, 4.86751],[52.36085, 4.86744],
[52.3608, 4.86722],[52.36077, 4.86708],[52.36072, 4.86689],        
[52.36067, 4.86662],[52.36063, 4.8665],[52.36062, 4.86643],
[52.36056, 4.86613],[52.36045, 4.86569],[52.36042, 4.86555],
[52.36048, 4.86553],[52.36061, 4.86546],[52.3607, 4.86542],
[52.36079, 4.86537],[52.36086, 4.86532],[52.36092, 4.86528],
[52.36102, 4.8652],[52.3612, 4.8651],[52.36138, 4.86499],
[52.36149, 4.86491],[52.36166, 4.8648],[52.36187, 4.86466],
[52.36197, 4.86463],[52.36227, 4.8644],[52.36236, 4.86433],
[52.36242, 4.86432]]


In [59]:
walkingroute1(coordinates)

In [58]:
def walkingroute1(coordinates):
    popup1 = 'start/finish'
    popup2 = 'Shiny Hopscotch'
    popup3 = 'LAB111'
    popup4 = 'Streetart Frankey'
    popup5 = "Ten Kate Markt"
    popup6 = "Picking garden" 
    if coordinates != False:
        start_coord = coordinates[0]
        end_coord = coordinates[len(coordinates) - 1]

        map = folium.Map(location=map_start(start_coord, end_coord),
                    zoom_start=15)
        
        folium.PolyLine(coordinates,
                        color='red',
                        weight=5,
                        opacity=0.8).add_to(map)
        
        folium.Marker(location=start_coord, popup=f'Start Location - <b>{popup1}</b>', tooltip=f'<strong>{popup1}</strong>', icon=folium.Icon(color='red', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=(52.3628251,4.8661345), popup=f'Shiny Hopscotch <b>{popup2}</b>', tooltip=f'<strong>{popup2}</strong>', icon=folium.Icon(color='gray', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=(52.363554,4.866921), popup=f'An old labratorium, what is currently housing a moviehouse showing classic, cult and new movies <b>{popup3}</b>', tooltip=f'<strong>{popup3}</strong>', icon=folium.Icon(color='gray', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=end_coord, popup=f'End Location - <b>{destination}</b>', tooltip=f'<strong>{destination}</strong>', icon=folium.Icon(color='red', icon='flag-checkered', prefix = 'fa')).add_to(map)
        folium.Marker(location=(52.363543, 4.866494), popup=f'Streetart Frankey is showing al lot of little art pieces in this street (try to find them) <b>{popup4}</b>', tooltip=f'<strong>{popup4}</strong>', icon=folium.Icon(color='gray', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=(52.3665673,4.8671821), popup=f'One of the coziest markets in Amsterdam next door is the foodhallen where you can get appetizers from all over the world <b>{popup5}</b>', tooltip=f'<strong>{popup5}</strong>', icon=folium.Icon(color='gray', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=(52.3641002,4.8656738), popup=f'Picking garden <b>{popup6}</b>', tooltip=f'<strong>{popup6}</strong>', icon=folium.Icon(color='gray', icon='map-pin', prefix = 'fa')).add_to(map)
        
        
        map.save("map.html")
        display(map)
    else:
        return "Could not process map"